In [ ]:
# ANALIZZARE UN MASSIMO DI 10 FILE
# Utilizzare poi merging_results_apa12.ipynb per unire i risultati

#DONE:
# Energy +5 GeV - 27367 : first 71 files analized
# Energy +2 GeV - 27355 : first 70 files analized
# Energy +1 GeV - 27343 : first 70 files analized
# Energy +3 GeV - 27361: first 70 files analized
# # Energy +5 GeV - 27374: first 70 files analized

In [ ]:
import importlib
import waffles.np04_analysis.lightyield_vs_energy.scripts.utils as utils_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils import *

importlib.reload(utils_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils import *

from tqdm.notebook import tqdm

In [ ]:
template_directory = "/afs/cern.ch/work/a/anbalbon/private/waffles/src/waffles/np04_analysis/lightyield_vs_energy/scripts/Renan_scripts/v_27xxx_fix_avg"
output_dir = "/afs/cern.ch/work/a/anbalbon/private/waffles/src/waffles/np04_analysis/lightyield_vs_energy/output/apa1_vs_apa2"
rucio_hdf5_dir = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio"
# rucio_hdf5_dir = "/eos/user/a/anbalbon/reading_rucio"

In [ ]:
energy = 7
run_number = 27374
reading_type = 'single' #["single", "merged"]

if reading_type == 'single': #reading single hdf5 files and merging it, without saving the merged file
    event_type = 'no_selection' # ["beam_only", "no_selection"] 
    n_files_start = 65
    n_files_stop = 70

elif reading_type == 'merged': #reading merged pkl file, produced by hdf5_run_merging_NEW.py
    FS_merged_filepath = ''
    ST_merged_filepath = ''
    
    # 3 GeV 
    # FS: /afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/run027361_full_streaming_no_selection_1st.pkl
    # ST: /afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/run027361_self_trigger_no_selection_1st.pkl
    
    # 5 GeV 
    # ST: /afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027367run027367_self_trigger_no_selection_51files.pkl

In [ ]:
# Lettura file SELF-TRIGGER

if reading_type == 'merged': 
    print("Loading Self-Trigger Wfset...", end='')
    with open(ST_merged_filepath, 'rb') as f:
        ST_wfset = pickle.load(f) 
        files_read = []
        print('\t --> \t ST done\n')  
else:
    ST_wfset, files_read = reading_merge_hdf5(run_number, 'self_trigger', event_type, rucio_hdf5_dir,  n_files_start, n_files_stop)


In [ ]:
# Lettura file FULL-STREAMING

if reading_type == 'merged': 
    print("Loading Full-Streaming Wfset...", end='')
    with open(FS_merged_filepath, 'rb') as f:
        FS_wfset = pickle.load(f) 
        files_read = []
        print('\t --> \t FS done\n')
else:
    FS_wfset, files_read = reading_merge_hdf5(run_number, 'full_streaming', event_type, rucio_hdf5_dir, n_files_start, n_files_stop)


In [ ]:

# Preparing wfset
time_info_dic = {}
wfset_dic = {}
for trigger_mode, apa, wfset in zip(['full_streaming', 'self_trigger'], [1, 2], [FS_wfset, ST_wfset]):
# for trigger_mode, apa, wfset in zip(['full_streaming'], [1], [FS_wfset]):
# for trigger_mode, apa, wfset in zip(['self_trigger'], [2], [ST_wfset]):

    # 1
    # Remove baseline and invert waveforms to have positive peaks (subtract_baseline_invert function) 
    baseline_analysis_label = 'baseliner'
    baseline_input_parameters = IPDict({'baseline_limits': [0, 50], 
                                            'std_cut': 3., 
                                            'type': 'mean'})    
    baseline_checks_kwargs = IPDict({'points_no': wfset.points_per_wf})
    _ = wfset.analyse(baseline_analysis_label, WindowBaseliner, baseline_input_parameters, checks_kwargs=baseline_checks_kwargs, overwrite=True)
    wfset.apply(subtract_baseline_invert, baseline_analysis_label, inversion = True, show_progress=False)


    # 2
    # Search for peaks (MyAnaPeak_NEW wfana)
    peak_analysis_class = MyAnaPeak_NEW
    peak_analysis_label = 'finding_peaks'
    peak_input_parameters = IPDict(baseline_limits= [0, 50],
                                    n_std = 5,
                                    peak_distance= 500,
                                    beam_timeoffset_limits = [-40, -20],
                                    signal_sign = "positive"
                                    )
    peak_checks_kwargs = IPDict({'points_no': wfset.points_per_wf})
    _ = wfset.analyse(label=peak_analysis_label ,analysis_class=peak_analysis_class, input_parameters=peak_input_parameters, checks_kwargs = peak_checks_kwargs, overwrite=True)


    # 3
    # Select beam waveforms: a waveform peak must be within a timeoffset range (), only one peak must satify this condition per waveform! (beam_filter filter function)
    wfset = WaveformSet.from_filtered_WaveformSet(wfset, beam_filter, peak_analysis_label)
    

    # 4
    # Creating list of offset time and absolute peal time 
    offset_list = []
    beam_peak_absolute_time_list = []
    for wf in wfset.waveforms:
        offset_list.extend(wf.analyses[peak_analysis_label].result['time_offset'])
        beam_peak_absolute_time_list.extend(wf.analyses[peak_analysis_label].result['beam_peak_absolute_time'])
    time_info_dic[trigger_mode] = {'offset_list': offset_list, 'beam_peak_absolute_time_list': beam_peak_absolute_time_list}

    #save wfset
    wfset_dic[apa] = wfset

print(f"FS wfset length --> BEFORE: {len(FS_wfset.waveforms)} \t AFTER: {len(wfset_dic[1].waveforms)}")
print(f"ST wfset length --> BEFORE: {len(ST_wfset.waveforms)} \t AFTER: {len(wfset_dic[2].waveforms)}")

In [ ]:
# + cut FS waveform around the peak (cut_full_streaming_window fuction)
wfset_dic[1].apply(cut_full_streaming_window, analysis_label = peak_analysis_label ,show_progress=False)

In [ ]:
# Saving wfset and list of read files
output_analysis_folder = f"{output_dir}/{energy}GeV/{n_files_start}_to_{n_files_stop}"
os.makedirs(output_analysis_folder, exist_ok=True)

print(f"Saving full-streaming wfset... ", end='')
with open(f"{output_analysis_folder}/FS_wfset.pkl", "wb") as f:
    pickle.dump(wfset_dic[1], f)
    print('done! \n')

print(f"Saving self-trigger wfset... ", end='')
with open(f"{output_analysis_folder}/ST_wfset.pkl", "wb") as f:
    pickle.dump(wfset_dic[2], f)
    print('done! \n')

print(f"Saving read_files info...", end='')
with open(f"{output_analysis_folder}/files_read.txt", "w", encoding="utf-8") as f:
    for item in files_read:
        f.write(f"{item}\n")
    print('done! \n')

In [ ]:
# 5
# Search for Full-streaming Self-trigger matching indentifing trigger times (trigger_time_list function)
all_beam_peak_absolute_time = sorted(time_info_dic['full_streaming']['beam_peak_absolute_time_list'] + time_info_dic['self_trigger']['beam_peak_absolute_time_list'])
print('Number of beam peaks before removing close values: ', len(all_beam_peak_absolute_time))
print('Number of beam peaks before removing close values (FS): ', len(time_info_dic['full_streaming']['beam_peak_absolute_time_list']))
print('Number of beam peaks before removing close values (ST): ', len(time_info_dic['self_trigger']['beam_peak_absolute_time_list']))
print()

delta_ts = 100 #minimum timeticks between two triggers 
trigger_time_list = trigger_time_searching(all_beam_peak_absolute_time, delta=delta_ts)
print("Number of beam peaks after removing close values (total): ", len(trigger_time_list))

In [ ]:
# 6
# Do a for cycle on each trigger time to extrapolate the mean number of PE for APA 1 and APA 2 

photoelectron_dict = {1 : {'channel_dic': [], 'list':[], 'mean':[], 'std':[], 'n_events':[]}, 2 : {'channel_dic': [], 'list':[], 'mean':[], 'std':[], 'n_events':[]}, 'trigger time' : []}

for i_trigger in tqdm(range(len(trigger_time_list)), desc="Processing triggers"):
    trigger_time = trigger_time_list[i_trigger]

    # 6.1
    # Select waveforms associated to this trigger timestamp (timestamp_filter filter function)
    # both wfset must exist!!!

    try:
        FS_trigger_wfset = WaveformSet.from_filtered_WaveformSet(wfset_dic[1], timestamp_filter, peak_analysis_label, trigger_time, delta_ts)
    except Exception as e:
        print(f'Error in FULL-STREAMING trigger wfset filter: {e}')
        continue
        
    try:
        ST_trigger_wfset = WaveformSet.from_filtered_WaveformSet(wfset_dic[2], timestamp_filter, peak_analysis_label, trigger_time, delta_ts)
    except Exception as e:
        print(f'{i_trigger} trigger --> Error in SELF-TRIGGER trigger wfset filter: {e}')
        continue


    for trigger_mode, apa, wfset in zip(['full_streaming', 'self_trigger'], [1, 2], [FS_trigger_wfset, ST_trigger_wfset]):
    # for trigger_mode, apa, wfset in zip(['full_streaming'], [1, 2], [FS_trigger_wfset]):
    # for trigger_mode, apa, wfset in zip(['self_trigger'], [1, 2], [ST_trigger_wfset]):

        # 6.2
        # Apply convolution to evaluate the number of photoelectrons of each waveform (MyAnaConvolution wfana)
        convolution_analysis_class = MyAnaConvolution
        convolution_analysis_label = 'searching_pe'
        convolution_input_parameters = IPDict(df_template = reading_template(template_directory),
                                                tick = 265) #???
        convolution_checks_kwargs = IPDict({'points_no': wfset.points_per_wf})
        _ = wfset.analyse( label=convolution_analysis_label, analysis_class=convolution_analysis_class, input_parameters=convolution_input_parameters, checks_kwargs=convolution_checks_kwargs, overwrite=True )
        
        # 6.3
        # Create a list with all n_pe and compute the average number
        dic_pe = {}
        list_pe = []
        grid_apa = ChannelWsGrid(APA_map[apa], wfset, compute_calib_histo=False)
        for run, run_dic in wfset.available_channels.items():
            for endpoint, channel_list in run_dic.items():
                if endpoint not in dic_pe:
                    dic_pe[endpoint] = {}
                for channel in channel_list:
                    dic_pe[endpoint][channel] = {}
                    try:
                        for i in range(len(grid_apa.ch_wf_sets[endpoint][channel].waveforms)):
                            list_pe.append(grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['n_pe'])
                            dic_pe[endpoint][channel]['n_pe'] = grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['n_pe']
                            dic_pe[endpoint][channel]['e_n_pe'] = grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['e_n_pe']
                            dic_pe[endpoint][channel]['fit_params'] = grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['fit_params']
                            dic_pe[endpoint][channel]['fit_errors'] = grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['fit_errors']
                            dic_pe[endpoint][channel]['r_squared'] = grid_apa.ch_wf_sets[endpoint][channel].waveforms[i].analyses[convolution_analysis_label].result['r_squared']
                    except Exception as e:
                        print(f'{i_trigger} trigger --> Error in grid_apa.ch_wf_sets[endpoint][channel]: {e}')
        
        list_pe = [x for x in list_pe if not math.isnan(x)]

        for endpoint in list(dic_pe.keys()):
            for channel in list(dic_pe[endpoint].keys()):
                if math.isnan(dic_pe[endpoint][channel]['n_pe']):
                    del dic_pe[endpoint][channel]
            if len(dic_pe[endpoint]) == 0:
                del dic_pe[endpoint]
        
        if len(list_pe):
            photoelectron_dict[apa]['list'].append(list_pe)
            photoelectron_dict[apa]['channel_dic'].append(dic_pe)
            photoelectron_dict[apa]['mean'].append(np.mean(list_pe))
            photoelectron_dict[apa]['std'].append(np.std(list_pe)) 
            photoelectron_dict[apa]['n_events'].append(len(list_pe))
        else: 
            photoelectron_dict[apa]['list'].append([])
            photoelectron_dict[apa]['channel_dic'].append({})
            photoelectron_dict[apa]['mean'].append(np.nan)
            photoelectron_dict[apa]['std'].append(np.nan) 
            photoelectron_dict[apa]['n_events'].append(0)

        
    photoelectron_dict['trigger time'].append(trigger_time)
    print(f"{i_trigger} trigger --> FS: {photoelectron_dict[1]['n_events'][-1]} events, mean pe = {photoelectron_dict[1]['mean'][-1]:.0f} +/- {photoelectron_dict[1]['std'][-1]:.0f}\t  ST: {photoelectron_dict[2]['n_events'][-1]:.0f} events, mean pe = {photoelectron_dict[2]['mean'][-1]:.0f} +/- {photoelectron_dict[2]['std'][-1]:.0f}")
        


In [ ]:
# Saving info in a json (all deconvolution info)
with open(f"{output_analysis_folder}/photoelectron_dic_{energy}GeV.json", "w", encoding="utf-8") as f:
    json.dump(prepare_for_json(photoelectron_dict), f, indent=4) 


In [ ]:
# Saving info as Dataframe

rows = []
for i in range(len(photoelectron_dict['trigger time'])):
    row = {
        'trigger_time': photoelectron_dict['trigger time'][i],

        'apa1_list':       photoelectron_dict[1]['list'][i],
        'apa1_mean':       photoelectron_dict[1]['mean'][i],
        'apa1_std':        photoelectron_dict[1]['std'][i],
        'apa1_n_events':   photoelectron_dict[1]['n_events'][i],

        'apa2_list':       photoelectron_dict[2]['list'][i],
        'apa2_mean':       photoelectron_dict[2]['mean'][i],
        'apa2_std':        photoelectron_dict[2]['std'][i],
        'apa2_n_events':   photoelectron_dict[2]['n_events'][i],
    }
    rows.append(row)

df_pe = pd.DataFrame(rows)
df_pe_name = f"photoelectron_dataframe_{energy}GeV.csv"
df_pe.to_csv(f"{output_analysis_folder}/{df_pe_name}", index=False)


In [ ]:
# If you wnat do read directly the dataset
df_pe_name = f"photoelectron_dataframe_{energy}GeV.csv"
df_pe = pd.read_csv(f"{output_analysis_folder}/{df_pe_name}")

In [ ]:
# 7
apa12_scatter(df_pe, energy, output_analysis_folder)

In [ ]:
# 8
apa1_hist_distribution(df_pe, energy, output_analysis_folder)

In [ ]:
# 9
apa2_hist_distribution(df_pe, energy, output_analysis_folder)

In [ ]:
# Offset histograms - FS
timeoffset_distribution_hist(time_info_dic['full_streaming']['offset_list'], 1, energy, output_analysis_folder, save = True)

In [ ]:
# Offset histograms - ST
timeoffset_distribution_hist(time_info_dic['self_trigger']['offset_list'], 2, energy, output_analysis_folder, save = True)